<a href="https://colab.research.google.com/github/lsraiva/sentinela-neural/blob/main/SENTINELA_NEURAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1 – Instalação da biblioteca Google GenAI (modelo Gemini)
!pip install google-genai

In [2]:
#2 – Importação da API key do usuário via Colab
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [3]:
#3 – Inicialização do cliente Gemini
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
#4 – Instalação das bibliotecas necessárias para processamento e IA
!pip install -q langchain
!pip install -q faiss-cpu
!pip install -q tiktoken
!pip install -q PyMuPDF
!pip install -q sentence-transformers

In [5]:
#5 – Atualização da biblioteca langchain-community
!pip install -U langchain-community

In [6]:
#6 – Imports principais: PDF, regex, IA generativa, embeddings, FAISS
import fitz  # PyMuPDF
import re
import google.generativeai as genai
from google.colab import files
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [7]:
#7 – Configuração da chave de API para uso do Gemini
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [8]:
#8 – Criação do modelo de embeddings com HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


<ipython-input-8-8ec4963e1400>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

In [9]:
#9 Download e leitura automática do PDF do Drive (DIAO)
import requests

# Link direto para download
url = "https://drive.google.com/uc?export=download&id=1TBNzDSuWzOnca3M884_QLfzyENxtVa-R"
pdf_path = "DIAO.pdf"

# Baixa e salva o PDF localmente
response = requests.get(url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

# Lê o PDF com PyMuPDF
doc_text = ""
with fitz.open(pdf_path) as pdf:
    for page in pdf:
        doc_text += page.get_text()

print("Documento carregado automaticamente com sucesso!")

Documento carregado automaticamente com sucesso!


In [10]:
#10 Divisão do texto com base nos títulos de artigos (ex: B 01.147 – Ameaça)
pattern = r'(B\s\d{2}\.\d{3}\s[-–]\s[^\n]+)'
splits = re.split(pattern, doc_text)

chunks = []
for i in range(1, len(splits), 2):
    titulo = splits[i].strip()
    conteudo = splits[i+1].strip()
    bloco = f"{titulo}\n{conteudo}"
    chunks.append(bloco)

print(f"{len(chunks)} blocos criados com base em cabeçalhos.")


144 blocos criados com base em cabeçalhos.


In [11]:
#11 Indexação FAISS com todos os blocos temáticos
vectorstore = FAISS.from_texts(chunks, embedding_model)
print("Indexação com FAISS finalizada.")

Indexação com FAISS finalizada.


In [12]:
#12 Função para buscar contexto relevante
def buscar_contexto_relevante(pergunta, k=6):
    resultados = vectorstore.similarity_search(pergunta, k=k)
    return "\n\n".join([r.page_content for r in resultados])


In [13]:
#13 Função para gerar resposta com Gemini + complemento técnico

def responder_com_rag(pergunta):
    contexto = buscar_contexto_relevante(pergunta)

    # Constrói o histórico da conversa (últimos 2 turnos)
    historico_formatado = ""
    for i in range(max(0, len(historico) - 4), len(historico), 2):
        historico_formatado += f"Usuário: {historico[i]}\n"
        if i + 1 < len(historico):
            historico_formatado += f"Assistente: {historico[i + 1]}\n"

    prompt = f"""
Você é um assistente jurídico-operacional da PMMG.

Converse com base no histórico abaixo e no documento institucional (DIAO). Use linguagem clara, objetiva e com base técnica.

Histórico:
{historico_formatado}

Nova pergunta: {pergunta}

Documento relevante:
{contexto}
"""

    resposta = genai.GenerativeModel(model_name=MODEL_ID).generate_content(prompt).text

    # Complemento automático para tema específico
    if "ameaça" in pergunta.lower() and "vias de fato" in pergunta.lower():
        complemento = "\n\n📌 Nota técnica: Segundo o Código Penal Brasileiro, **ameaça (art. 147)** é classificada como **crime de menor potencial ofensivo**, enquanto **vias de fato (art. 21 da LCP)** é uma **contravenção penal**. Portanto, **ameaça é juridicamente mais grave que vias de fato**."
        resposta += complemento

    # Armazena a interação no histórico
    historico.append(pergunta)
    historico.append(resposta)

    return resposta


In [ ]:
#14 Loop de teste interativo

historico = []  # Armazena os últimos turnos da conversa

while True:
    pergunta = input("Digite sua dúvida (ou 'sair' para encerrar): ")
    if pergunta.lower() == 'sair':
        break
    resposta = responder_com_rag(pergunta)
    print("\n📌 Resposta:", resposta, "\n")